# Environment setup

## Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate deep_research

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=deep_research --display-name "deep_research"
```
Once this is done, you can select "deep_research" from the Kernel → Change Kernel menu in Jupyter or VS Code.

## Ollama setup

In this project we use the `llama3.2:3b` and `deepseek-r1:8b` models. You can try other smaller or larger reasoning LLMs such as `qwen2.5:3b-instruct` or `phi4-mini` to compare performance. Explore available models here: https://ollama.com/library.

```bash
ollama pull llama3.2:3b
ollama pull deepseek-r1:8b
# Additional small reasoning models to compare
# ollama pull qwen2.5:3b-instruct
# ollama pull phi4-mini

```

`ollama pull` downloads the model so you can run it locally without API calls.

---  
# A Deep Research Agent

A deep-research agent pairs a reasoning model (e.g., deepseek-r1) with external tools for web search and retrieval. We will follow the ReAct pattern: the model writes short thoughts, decides when to call tools, reads observations, and continues reasoning until it can answer or reaches a step limit.

We now combine a **search tool** with a reasoning model (e.g., `deepseek-r1`) in a multi-step setup. We follow the *ReAct* pattern (reason → tool → observation):

1. The model reasoins and decides to use tools
2. The agent searches and feed condensed snippets back as context
3. Iterate until the model answers or hits a step limit

We use `AgentType.OPENAI_FUNCTIONS`, which hides the loop inside the LangChain agent.

In [ ]:
from ddgs import DDGS
from langchain.tools import Tool


def ddg_search(query: str, k: int = 5) -> str:
    """Basic DuckDuckGo web search that returns a concatenated text snippet."""
    with DDGS() as ddgs:
        results = [hit["body"] for hit in ddgs.text(query, max_results=k)]
    return "\n".join(results)

search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the public web. Input: a plain English query. Returns: concatenated snippets."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

MODEL = "deepseek-r1:8b"
question = "What are the best resources to learn machine learning in 2025?"

# Step 1: Initialize the reasoning model via ChatOllama
llm = ChatOllama(model=MODEL, temperature=0.2)

# Step 2: Build the agent with tool access (DuckDuckGo Search) and function-calling interface (initialize_agent)
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

# Step 3: Ask a query and let the agent search + reason to produce an answer
result = agent.invoke({"input": question})
print(result["output"])



> Entering new AgentExecutor chain...
<think>
Okay, user is asking about the best resources to learn machine learning for 2025. That's an interesting timeframe - they're looking ahead two years, which suggests they want future-proof knowledge rather than just current trends. 

First, I should acknowledge that predicting exact educational needs two years in advance is tricky because ML evolves rapidly. But some foundational concepts and practical skills will likely remain relevant regardless of technological shifts. The user probably wants to know what's most valuable now but with an eye toward long-term career sustainability.

Hmm... they didn't specify their background or goals, which makes this broad. Are they a complete beginner? A developer looking to pivot? Or someone with math/stats knowledge wanting hands-on skills? Since they didn't say, I should cover multiple learning paths comprehensively.

I notice they're asking about "resources" plural - not just courses but books, docu

# Multi-agent Deep Research
Instead of a single multi-step agent, you can design multiple collaborating agents such as a Planner, Searcher, Summarizer, and Verifier that pass information and refine each other’s outputs. This setup improves robustness, diversity of reasoning, and division of labor.

Below is a simple setup with 2–3 agents that share goals and messages, for example Planner → Researcher → Writer.

In [ ]:
import json
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.schema import SystemMessage, HumanMessage
from ddgs import DDGS
import langchain
print(langchain.__version__) 
# LangChain 0.3.26 is quite recent (released in late 2024). 
# With version 0.3.26, when you use AgentType.OPENAI_FUNCTIONS with a local model that doesn't support function calling


0.3.26


In [ ]:
# --- CONFIGURATION ---
# We use the same model for all agents, but with different system prompts.
# You could swap these: e.g., use a larger model for the Planner 
# and a faster one for the Researcher.
MODEL_NAME = "deepseek-r1:8b"

In [ ]:
# --- 1. DEFINE THE TOOLS ---
def ddg_search(query: str, k: int = 5) -> str:
    """Performs a web search and returns concatenated snippets."""
    print(f"🔍 DDG_SEARCH CALLED with query: '{query}'") 
    try:
        with DDGS() as ddgs:
            results = [hit["body"] for hit in ddgs.text(query, max_results=k)]
            print(f"✅ Got {len(results)} real results") # Log the number of results retrieved = k = 5
            return "\n".join(results)
    except Exception as e:
        print(f"❌ Search error: {e}")
        return f"Search error: {e}"

search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the web. Input: specific search query."
)

In [ ]:
# --- 2. DEFINE THE AGENTS ---

class PlannerAgent:
    def __init__(self):
        self.llm = ChatOllama(model=MODEL_NAME, temperature=0.2)
    
    def plan(self, user_query):
        """Breaks the user query into a list of specific research questions."""
        print(f"🔵 [Planner] Analyzing request: '{user_query}'...")
        
        prompt = f"""
        You are a Research Planner. Your goal is to break down a complex user question into 3 distinct, specific search queries that a researcher needs to investigate to answer the question comprehensively.
        
        User Question: "{user_query}"
        
        Return ONLY a JSON list of strings, like this format:
        ["query 1", "query 2", "query 3"]
        Do not add any other text.
        """
        response = self.llm.invoke(prompt).content
        
        # Simple cleanup to ensure we get a list
        try:
            # Extract list from text (in case model adds extra chars)
            start = response.find('[')
            end = response.rfind(']') + 1
            json_str = response[start:end]
            plan = json.loads(json_str)
            print(f"   -> Plan created: {plan}")
            return plan
        except Exception as e:
            print(f"   -> Planning failed, falling back to original query. Error: {e}")
            return [user_query]

class ResearcherAgent:
    def __init__(self):
        # We use an Agent with tools here because the researcher needs to actually *do* the search
        self.llm = ChatOllama(model=MODEL_NAME, temperature=0.5)
        self.agent = initialize_agent(
            tools=[search_tool],
            llm=self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # a compatible agent type
            verbose=True, # enables the display of the agent's internal thought process and actions in the console
            handle_parsing_errors=True # helps with any JSON parsing issues from the model
        )
            # agent=AgentType.OPENAI_FUNCTIONS, 
            #
            # """ 
            # Problem: OPENAI_FUNCTIONS requires function-calling capabilities that most local models don't support.
            # 
            # The problem When you use AgentType.OPENAI_FUNCTIONS with DeepSeek-R1. 
            # The agent framework tells the model "you have access to a search tool"
            # The model pretends to use it in its reasoning
            # But it never actually calls your function
            # Instead, it generates fake results from its training data
            # """
        
            #    verbose=True,
            #    -> Searching: 'Query 1'
            #    [agent verbose output]
            #    -> Agent returned: <class 'dict'>

            #    -> Searching: 'Query 2'  
            #    [agent verbose output]
            #    -> Error searching 'Query 2': An output parsing error...

            #    -> Searching: 'Query 3'
            #    [agent verbose output]
            #    -> Error searching 'Query 3': An output parsing error...

    def research(self, queries):
        """Takes a list of queries and executes them one by one."""
        notes = []
        print(f"🟡 [Researcher] Starting investigation...")
        
        for q in queries:
            print(f"   -> Searching: '{q}'")
            
            try:
                # The agent decides how to use the search tool for this query
                result = self.agent.invoke({"input": f"Find detailed information about: {q}"})
                print(f"   -> Agent returned: {type(result)}")
                notes.append(f"Query: {q}\nFindings: {result['output']}")
            except Exception as e:
                print(f"   -> Error searching '{q}': {e}")
        
        return "\n\n".join(notes)

class WriterAgent:
    def __init__(self):
        self.llm = ChatOllama(model=MODEL_NAME, temperature=0.3)
    
    def write(self, original_query, research_notes):
        """Synthesizes the research notes into a final answer."""
        print(f"🟢 [Writer] Synthesizing report...")
        
        prompt = f"""
        You are a technical writer. You have been given research notes from a field researcher.
        Your task is to answer the User's original question using ONLY these notes.
        
        User Question: "{original_query}"
        
        Research Notes:
        {research_notes}
        
        Write a comprehensive, well-structured report. Cite the specific findings from the notes where applicable.
        """
        response = self.llm.invoke(prompt).content
        return response

In [35]:
# --- 3. ORCHESTRATION ---

def run_collaborative_research(user_query):
    # Initialize agents
    planner = PlannerAgent()
    researcher = ResearcherAgent()
    writer = WriterAgent()
    
    # Step 1: Plan
    search_queries = planner.plan(user_query)
    
    # Step 2: Research
    # (This iterates through the plan. You could also parallelize this step like in the previous code!)
    raw_notes = researcher.research(search_queries)
    
    # Step 3: Write
    final_report = writer.write(user_query, raw_notes)
    
    print("\n" + "="*40)
    print("FINAL REPORT")
    print("="*40 + "\n")
    print(final_report)
    return final_report

In [36]:
# --- 4. EXECUTION ---
if __name__ == "__main__":
    query = "Compare the top 3 open-source LLMs released in late 2024 and 2025 suitable for coding."
    run_collaborative_research(query)

🔵 [Planner] Analyzing request: 'Compare the top 3 open-source LLMs released in late 2024 and 2025 suitable for coding.'...
   -> Plan created: ['Top 3 open-source LLMs released between December 2024 and March 2025', 'Open-source LLMs optimized for coding tasks released between December 2024 and March 2025', 'Best open-source LLMs for coding released between December 2024 and March 2025']
🟡 [Researcher] Starting investigation...
   -> Searching: 'Top 3 open-source LLMs released between December 2024 and March 2025'


> Entering new AgentExecutor chain...

Thought: The user is asking for the top 3 open-source large language models (LLMs) released between December 2024 and March 2025. I need to find reliable sources that list recent LLM releases in that timeframe. Since March 2025 is in the future, I'll focus on models released in December 2024 and any early 2025 releases. I should search for recent announcements from major LLM developers like Meta, Mistral, OpenAI, and others.

Action: I

## 🎉 Congratulations!

* You have built a **deep-research agent**: reasoning model like deep-seek r1 + ReAct-style agent + tool use (web search)
* You have added a search tool, and extended the deep-research to a multi-agent system: many agents.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.